In [9]:
import torch
import matplotlib.pyplot as plt
import numpy as np 
import pycolmap
from pathlib import Path
from matplotlib import image as mpimg
from src.geometry.point_transformation import *
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
south_building_model = torch.load("../results/south-building/ckpts/ckpt_29999_rank0.pt", map_location="cpu", weights_only=True)

In [7]:
recon_path = Path("../data/south-building/0")
images_folder = Path("../data/south-building/images")

In [6]:
device=torch.device("cpu")

In [13]:
reconstruction = pycolmap.Reconstruction(recon_path)
pcd = o3d.io.read_point_cloud("../data/south-building/sparse.ply")
recon_points = torch.tensor(np.asarray(pcd.points), device=device)

img_id = 1
img = reconstruction.images[img_id]
cam_id = 1
image = mpimg.imread(images_folder / f'{img.name}')

extrinsic_matrix = get_extrinsic_params(img.cam_from_world)
f, cx, cy = reconstruction.cameras[cam_id].params
f_pt = torch.scalar_tensor(f, device=device, dtype=torch.float32)
width, height = reconstruction.cameras[cam_id].width, reconstruction.cameras[cam_id].height
znear, zfar = 1, 5
intrinsic_matrix = get_intrinsic_opengl_params(f, f, height, width, zfar=zfar, znear=znear)
W = torch.tensor(extrinsic_matrix[:3, :3].T, device=device, dtype=torch.float32) # viewing transformation

extrinsic_matrix_pt = torch.tensor(extrinsic_matrix, device=device, dtype=torch.float32)
intrinsic_matrix_pt = torch.tensor(intrinsic_matrix, device=device, dtype=torch.float32)

image_pt = torch.tensor(image / 255.0, device=device)

In [14]:
homogeneous_points = convert_to_homogenous(recon_points)
camera_coordinates = homogeneous_points @ extrinsic_matrix_pt.T
clip_coordinates = camera_coordinates @ intrinsic_matrix_pt.T
point_ids = cull_coordinates_ids(clip_coordinates, camera_coordinates, zfar=zfar, znear=znear)
ndc_coordinates = to_ndc_coordinates(clip_coordinates[point_ids])
screen_coordinates = to_screen_coordinates(ndc_coordinates, width, height, zfar, znear)

TypeError: unsupported operand type(s) for @: 'numpy.ndarray' and 'Tensor'